## Load data from database export and reformat

In [ ]:
import pandas as pd
import numpy as np
import pickle
import plotly.express as px

In [ ]:
# create figure dictionary

fig_dict = {}

In [ ]:
# Load data from csv, sql query that generated this csv is queries/user_cohort_table_query.sql
df = pd.read_csv('data/user_activity_cohort_table_2023-09-09T14_17_51.117818Z.csv', low_memory=False)

In [ ]:
# Columns to keep
c = [
    'UserId',
    'UserSlug',
    'Expense Activities',
    'Expense Hosts',
    'Expense to Own Collective Activities',
    'Expense to Own Collective Hosts',
    'Host Admin Activities',
    'Host Admin Hosts',
    'Collective Admin Activities',
    'Collective Admin Hosts',
    'Direct Contributions',    
    'Direct Contributions Hosts',
    'Direct Contributions Recipients',
    'Collective Contributions',
    'Collective Contributions Hosts',
    'Collective Contributions Recipients',
    'Contributions to Own Collective',
    'Contributions to Own Collective Hosts',
    'Contributions to Own Collective Recipients',
    'Contributions Via Host',
    'Contributions Via Host Hosts',
    'Contributions Via Host Recipients',
    'Event Orders',
    'Event Orders Hosts',
    'Event Orders Recipients',
    'Organization Contributions',
    'Organization Contributions Hosts',
    'Organization Contributions Recipients',
    'Virtual Card Purchases',
    'Virtual Card Purchases Hosts'
]
df = df[c]

In [ ]:
# Evaluate lists of collectives

stringlists = [
'Collective Contributions Hosts',
'Collective Contributions Recipients',
'Direct Contributions Hosts',
'Direct Contributions Recipients',
'Contributions to Own Collective Hosts',
'Contributions to Own Collective Recipients',
'Contributions Via Host Hosts',
'Contributions Via Host Recipients',
'Organization Contributions Hosts',
'Organization Contributions Recipients',
'Event Orders Hosts',
'Event Orders Recipients',
'Expense Hosts',
'Expense to Own Collective Hosts',
'Host Admin Hosts',
'Collective Admin Hosts',
'Virtual Card Purchases Hosts'
]

for s in stringlists:
    df[s] = df[s].apply(lambda x: x[1:-1].split(', ') if type(x) == str else [])
    df[s] = df[s].apply(lambda x: [] if type(x) != list else x)
    df[s] = df[s].apply(lambda x: [i for i in x if i != ''])
    df[s].head()

In [ ]:
# convert all columns to snake case
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.rename(columns={'userid': 'user_id', 'userslug': 'user_slug'}, inplace=True)

## Preprocess data

In [ ]:
# Count all users with any activity in the period

all_users_df = df


In [ ]:
# drop scammy outliers and host admins with scripted activity

# users sudharaka-palamakumbura and znarf have already been removed from the dataset in the sql query

# if hon-community-thailand or heroes-of-newerth-community are in any of the lists, drop the row

df_processed = all_users_df.copy()
df_processed = df_processed[~df_processed['collective_contributions_hosts'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['collective_contributions_recipients'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['direct_contributions_hosts'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['direct_contributions_recipients'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['contributions_to_own_collective_hosts'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['contributions_to_own_collective_recipients'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['contributions_via_host_hosts'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['contributions_via_host_recipients'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['organization_contributions_hosts'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['organization_contributions_recipients'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['event_orders_hosts'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['event_orders_recipients'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['expense_hosts'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['expense_to_own_collective_hosts'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['host_admin_hosts'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]
df_processed = df_processed[~df_processed['collective_admin_hosts'].apply(lambda x: 'hon-community-thailand' in x or 'heroes-of-newerth-community' in x)]

In [ ]:
# Create columns for total numbers of contributions, expenses and activities

# Sum the number of Direct Contributions and Collective Contributions for each user
df_processed['total_contributions'] = df_processed['direct_contributions'] + df_processed['collective_contributions'] + df_processed['contributions_to_own_collective'] + df_processed['contributions_via_host'] + df_processed['organization_contributions'] + df_processed['event_orders']

# Sum the number of expense_activities, collective_expense_activities and Virtual Card Purchases for each user
df_processed['total_expenses'] = df_processed['expense_activities'] + df_processed['expense_to_own_collective_activities'] + df_processed['virtual_card_purchases']

# total number of activities
df_processed['total_activities'] = df_processed['total_contributions'] + df_processed['total_expenses'] + df_processed['host_admin_activities'] + df_processed['collective_admin_activities']


In [ ]:
# Only keep users with at least one activity of the types we're interested in

counted_users_df = df_processed[df_processed['total_activities'] > 0]


In [ ]:
# Define active users as those with total activities > threshold
threshold = 0
active_users_df = counted_users_df[counted_users_df['total_activities'] > threshold].copy()

In [ ]:
# Define higly active users as those with total activities > high_threshold
high_threshold = 11
highly_active_users_df = counted_users_df[counted_users_df['total_activities'] > high_threshold]


In [ ]:
# Calculate log of all activity counts

ac = [
    'expense_activities',
    'expense_to_own_collective_activities',
    'host_admin_activities',
    'collective_admin_activities',
    'direct_contributions',    
    'collective_contributions',
    'contributions_to_own_collective',
    'contributions_via_host',
    'organization_contributions',
    'event_orders',
    'virtual_card_purchases',
    'total_contributions',
    'total_expenses',
    'total_activities'
]

# Add new columns to df with log of activity counts, named with Log prefix
for a in ac:
    active_users_df[a + '_log'] = np.log(active_users_df[a] + 1)

In [ ]:
# Calculate normalized log scores for each activity type

# Create new columns with normalized log scores between 0 and 1, named with Norm prefix
for a in ac:
    active_users_df[a + '_norm_log'] = (active_users_df[a + '_log'] - active_users_df[a + '_log'].min()) / (active_users_df[a + '_log'].max() - active_users_df[a + '_log'].min())

In [ ]:
# Calculate score (0,1,2,3) of each user for each activity type based on normalized log

# Create new columns with score 0,1,2,3 based on norm log, named with _score suffix
# If norm log is 0, score is 0
# If norm log is between 0 and 0.33, score is 1
# If norm log is between 0.33 and 0.66, score is 2
# If norm log is between 0.66 and 1, score is 3
for a in ac:
    active_users_df[a + '_score'] = active_users_df[a + '_norm_log'].apply(lambda x: 0 if x == 0 else 1 if x < 0.33 else 2 if x < 0.66 else 3)

In [ ]:
# Calculate Yeo-Johnson power transform score

from sklearn.preprocessing import PowerTransformer, MinMaxScaler

pt = PowerTransformer(method='yeo-johnson', standardize=False)
pt_std = PowerTransformer(method='yeo-johnson', standardize=True)

for c in ac:
    # Transform the columns and add them to the dataframe with name <column name> Yeojohnson
    # And normalize the values between 0 and 1
    active_users_df[f'{c}_yeojohnson'] = pt_std.fit_transform(active_users_df[[c]])
    active_users_df[f'{c}_yeojohnson_scaled'] = MinMaxScaler().fit_transform(pt.fit_transform(active_users_df[[c]]))

In [ ]:
# Reorder columns, first column is user_id, second column is user_slug, rest of columns are in alphabetical order
active_users_df = active_users_df.reindex(sorted(active_users_df.columns), axis=1)
active_users_df = active_users_df[['user_id', 'user_slug'] + sorted(list(set(active_users_df.columns) - set(['user_id', 'user_slug'])))]
# reorder so that all columns that end with _host or _recipient are at the end
host_recipient_cols = [col for col in active_users_df.columns if col.endswith('_hosts') or col.endswith('_recipients')]
other_cols = [col for col in active_users_df.columns if col not in host_recipient_cols]

# Combine both lists to get the new order
new_order = other_cols + host_recipient_cols

# Reorder DataFrame based on this new order
active_users_df = active_users_df[new_order]

In [ ]:
# define columns for each activity type and their corresponding score columns

cols = [
    ("expense_activities", "expense_activities_score"), 
    ("expense_to_own_collective_activities", "expense_to_own_collective_score"),
    ("virtual_card_purchases", "virtual_card_purchases_score"),
    ("total_expenses", "total_expenses_score"),
    ("host_admin_activities", "host_admin_activities_score"),
    ("collective_admin_activities", "collective_admin_activities_score"),
    ("direct_contributions", "direct_contributions_score"),
    ("collective_contributions", "collective_contributions_score"),
    ("contributions_to_own_collective", "contributions_to_own_collective_score"),
    ("contributions_via_host", "contributions_via_host_score"),
    ("organization_contributions", "organization_contributions_score"),
    ("event_orders", "event_orders_score"),
    ("total_contributions", "total_contributions_score")
    ]

## Dataset stats

In [ ]:
# Count the number of users in different categories

all_users_number = len(all_users_df)
counted_users_number = len(counted_users_df)
active_users_number = len(active_users_df)
highly_active_users_number = len(highly_active_users_df)

expenses = all_users_df[all_users_df["expense_activities"] > 0]["expense_activities"]
expenses_to_own_collective = all_users_df[all_users_df["expense_to_own_collective_activities"] > 0]["expense_to_own_collective_activities"]
host_admin_activities = all_users_df[all_users_df["host_admin_activities"] > 0]["host_admin_activities"]
collective_admin_activities = all_users_df[all_users_df["collective_admin_activities"] > 0]["collective_admin_activities"]
direct_contributions = all_users_df[all_users_df["direct_contributions"] > 0]["direct_contributions"]
collective_contributions = all_users_df[all_users_df["collective_contributions"] > 0]["collective_contributions"]
contributions_to_own_collective = all_users_df[all_users_df["contributions_to_own_collective"] > 0]["contributions_to_own_collective"]
contributions_via_host = all_users_df[all_users_df["contributions_via_host"] > 0]["contributions_via_host"]
organization_contributions = all_users_df[all_users_df["organization_contributions"] > 0]["organization_contributions"]
event_orders = all_users_df[all_users_df["event_orders"] > 0]["event_orders"]
virtual_card_purchases = all_users_df[all_users_df["virtual_card_purchases"] > 0]["virtual_card_purchases"]

## Activities counted in dataset

**Expense activities:** collective.expense.created

**Virtual card activities:** virtualcard.purchase

**Host admin activities:** collective.approved, collective.rejected, collective.expense.paid, collective.expense.incomplete, collective.expense.scheduledForPayment, collective.virtualcard.added, collective.virtualcard.deleted, collective.virtualcard.request.rejected, collective.virtualcard.request.approved, collective.expense.putOnHold, collective.expense.releasedFromHold, collective.expense.reApprovalRequested, collective.expense.unscheduledForPayment, agreement.created, agreement.edited, collective.created

**Collective admin activities:** collective.apply, collective.core.member.edited, collective.core.member.removed, collective.expense.approved, collective.expense.unapproved, collective.update.created

**Contributor activities:** orders updated in period where status is "PAID", "ACTIVE" or "CANCELLED"

In [ ]:
# Print user activity counts

print(f'Number of unique users with any activity in the period: {all_users_number}')
print(f'Counted users, number of users with at least one counted activity: {counted_users_number}')
print(f'Active users, number of users with at least {threshold + 1} counted activities: {active_users_number}')
print(f'Highly active users, number of users with at least {high_threshold + 1} counted activities: {highly_active_users_number}')
print()
print(f'Counted activites: Host and collective admin activities, direct and collective expenses, virtual card purchases, and direct and collective contributions ')


In [ ]:
# Draw

data = dict(
    number=[all_users_number, counted_users_number, highly_active_users_number],
    stage=["All users with any activity ", "Users with +1 counted activity ", "Users with +12 counted activites "])


fig = px.funnel(data, x='number', y='stage', custom_data=['number'])
# do not show y axis title
fig.update_yaxes(title_text='')
fig.show()

fig_dict['active_users'] = {
    'fig': fig, 
    'title': 'User Activity Counts', 
    'description': 'Number of users with any activity, users with at least one counted activity, and users with at least 12 counted activities',
    'info': """
    <p>For 'all users' any user is counted who has any recorded activity in the activity log. This also includes signing in to the platform. For the 'counted activities' groups the following activities are counted and aggregated into activity classes:</p> <br>
    <ul>
    <li><b>Expense activities:</b> collective.expense.created, virtualcard.purchase </li>
    <li><b>Host admin activities:</b> collective.approved, collective.rejected, collective.expense.paid, collective.expense.incomplete, collective.expense.scheduledForPayment, collective.virtualcard.added, collective.virtualcard.deleted, collective.virtualcard.request.rejected, collective.virtualcard.request.approved, collective.expense.putOnHold, collective.expense.releasedFromHold, collective.expense.reApprovalRequested, collective.expense.unscheduledForPayment, agreement.created, agreement.edited, collective.created </li>
    <li><b>Collective admin activities:</b> collective.apply, collective.core.member.edited, collective.core.member.removed, collective.expense.approved, collective.expense.unapproved, collective.update.created</li>
    <li><b>Contributor activities:</b> Not counted from the activity log as we only want to capture activities that users do on the platform, not activities that are automatic. We define a contributor activity as making or updating an order, so we count orders that are updated by the user in the period with status orders updated in period where status is "PAID", "ACTIVE" or "CANCELLED".</li>
    </ul>
    <br>
    <p>Note that some activities are not counted because they are not specific to any user class, so do not help us to identify user classes. Examples of these are rejecting an expense and posting an expense comment.</p>
    """
    }

In [ ]:
# Print user stats for each activity type

print(f'Number of users with expenses to own collectives: {expenses_to_own_collective.shape[0]}')
print(f'         - {round(expenses_to_own_collective.shape[0]/all_users_number*100,2)}% of all users')
print(f'         - {round(expenses_to_own_collective.shape[0]/counted_users_number*100,2)}% of counted users')
print()
print(f'Number of users with expenses to other collectives: {expenses.shape[0]}')
print(f'         - {round(expenses.shape[0]/all_users_number*100,2)}% of all users')
print(f'         - {round(expenses.shape[0]/counted_users_number*100,2)}% of counted users')
print()
print(f'Number of users with virtual card purchases: {virtual_card_purchases.shape[0]}')
print(f'         - {round(virtual_card_purchases.shape[0]/all_users_number*100,2)}% of all users')
print(f'         - {round(virtual_card_purchases.shape[0]/counted_users_number*100,2)}% of counted users')
print()
print(f'Number of users with host admin activities: {host_admin_activities.shape[0]}')
print(f'         - {round(host_admin_activities.shape[0]/all_users_number*100,2)}% of all users')
print(f'         - {round(host_admin_activities.shape[0]/counted_users_number*100,2)}% of counted users')
print()
print(f'Number of users with collective admin activities: {collective_admin_activities.shape[0]}')
print(f'         - {round(collective_admin_activities.shape[0]/all_users_number*100,2)}% of all users')
print(f'         - {round(collective_admin_activities.shape[0]/counted_users_number*100,2)}% of counted users')
print()
print(f'Number of users with contributions via host: {contributions_via_host.shape[0]}')
print(f'         - {round(contributions_via_host.shape[0]/all_users_number*100,2)}% of all users')
print(f'         - {round(contributions_via_host.shape[0]/counted_users_number*100,2)}% of counted users')
print()
print(f'Number of users with contributions to own collective: {contributions_to_own_collective.shape[0]}')
print(f'         - {round(contributions_to_own_collective.shape[0]/all_users_number*100,2)}% of all users')
print(f'         - {round(contributions_to_own_collective.shape[0]/counted_users_number*100,2)}% of counted users')
print()
print(f'Number of users with organization contributions: {organization_contributions.shape[0]}')
print(f'         - {round(organization_contributions.shape[0]/all_users_number*100,2)}% of all users')
print(f'         - {round(organization_contributions.shape[0]/counted_users_number*100,2)}% of counted users')
print()
print(f'Number of users with collective contributions: {collective_contributions.shape[0]}')
print(f'         - {round(collective_contributions.shape[0]/all_users_number*100,2)}% of all users')
print(f'         - {round(collective_contributions.shape[0]/counted_users_number*100,2)}% of counted users')
print()
print(f'Number of users with direct contributions: {direct_contributions.shape[0]}')
print(f'         - {round(direct_contributions.shape[0]/all_users_number*100,2)}% of all users')
print(f'         - {round(direct_contributions.shape[0]/counted_users_number*100,2)}% of counted users')
print()
print(f'Number of users with event orders: {event_orders.shape[0]}')
print(f'         - {round(event_orders.shape[0]/all_users_number*100,2)}% of all users')
print(f'         - {round(event_orders.shape[0]/counted_users_number*100,2)}% of counted users')
print()

In [ ]:
# Generate markdown for the notebook

from IPython.display import Markdown as md

fr=2

md("# All analysis below concerns the subset of active users (n=%s) with at least %i core activities."%("{:,}".format(active_users_number),threshold+1))

In [ ]:
plot_yeojohnson = False
if plot_yeojohnson:
    # Using active_users_df, plot activity histograms and Yeo-Johnson transformation scatterplots for all activity types

    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Set the Seaborn style
    sns.set_style("darkgrid")

    # Create a figure and array of axes for the number of features
    num_features = len(cols)
    f, axarr = plt.subplots(num_features, 3, figsize=(15, 5*num_features))

    for idx, c in enumerate(cols):

        feature_title = c[0].replace('_', ' ').title()

        # Count the number of users with at least one activity of the type we're interested in
        num_users = active_users_df[active_users_df[c[0]] > 0].shape[0]

        # Show a Pie Chart of the number of users with at least one activity
        pie_labels = [feature_title, 'No ' + feature_title]
        axarr[idx, 0].pie([num_users, active_users_number - num_users], autopct='%1.1f%%', startangle=90)
        axarr[idx, 0].set_title(feature_title + ' Among Active Users')

        # change color of the first slice to orange
        axarr[idx, 0].patches[0].set_facecolor('orange')
        # change color of the second slice to grey
        axarr[idx, 0].patches[1].set_facecolor('grey')

        axarr[idx, 0].legend(pie_labels, loc='upper center', bbox_to_anchor=(0.5, 0.10))
        
        # Activity Histogram using Seaborn
        feature = active_users_df[active_users_df[c[0]] > 0][c[0]]
        sns.histplot(data=feature, bins=np.arange(1, 21, 1), edgecolor='black', ax=axarr[idx, 1], kde=False)
        axarr[idx, 1].set_xticks(np.arange(1, 21, 1))
        axarr[idx, 1].set_xticklabels(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20+'])
        axarr[idx, 1].set_title(feature_title)
        axarr[idx, 1].set_xlabel(f'Number of {feature_title}')
        axarr[idx, 1].set_ylabel('Number of Users')

        # Yeo-Johnson Transformation Scatterplot using Seaborn
        sns.scatterplot(x=active_users_df[c[0]], y=active_users_df[f'{c[0]}_yeojohnson'], ax=axarr[idx, 2])
        axarr[idx, 2].set_title(f'{feature_title} vs Yeo-Johnson Transformation')
        axarr[idx, 2].set_xlabel(feature_title)
        axarr[idx, 2].set_ylabel(f'Yeo-Johnson Transformation')


    # After setting your subplots
    plt.subplots_adjust(top=0.95)  # This reduces the height of the subplot area to 90% of the figure, leaving 10% at the top for the title.

    # Add space between the rows of subplots
    plt.subplots_adjust(hspace=0.5)

    plt.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

activity_figs = []

for idx, c in enumerate(cols):
    feature_title = c[0].replace('_', ' ').title()

    # Count the number of users with at least one activity of the type we're interested in
    num_users = active_users_df[active_users_df[c[0]] > 0].shape[0]

    # Initialize a 1x2 subplot with appropriate types
    fig = make_subplots(rows=1, cols=2, 
                        specs=[[{'type':'pie'}, {'type':'xy'}]])

    # Pie Chart of the number of users with at least one activity
    pie_data = pd.DataFrame({
        'Activity': [feature_title, 'No ' + feature_title],
        'Count': [num_users, active_users_number - num_users]
    })

    fig.add_trace(
        go.Pie(labels=pie_data['Activity'], values=pie_data['Count'], 
               marker=dict(colors=['orange', 'grey']), showlegend=False, hovertemplate='Users with %{label}: %{value}', name=''), 
        row=1, col=1
    )

    # Activity Histogram
    feature_data = active_users_df[active_users_df[c[0]] > 0][c[0]]
    # Group values of 20 or more together
    feature_data = feature_data.apply(lambda x: 20 if x >= 20 else x)

    fig.add_trace(
        go.Histogram(x=feature_data, marker_color='blue', showlegend=False,
                     xbins=dict(start=0.5, end=21, size=1), hovertemplate='Users with %{x} activities: %{y}', name=''), 
        row=1, col=2
    )

    fig.update_xaxes(tickvals=list(range(1, 21)), 
                     ticktext=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20+'], 
                     row=1, col=2,
                     title_text="Activities")

    fig.update_yaxes(title_text="Users", row=1, col=2)

    fig.show()

    activity_figs.append({
        'fig': fig,
        'title': feature_title,
        'description': f'Number of users with at least one {feature_title} and histogram of number of {feature_title} per user.',
        'info': f"""
        <p>For {feature_title} we count the number of users who have at least one {feature_title} in the period. We also plot a histogram of the number of {feature_title} per user.</p>
        <p>For the histogram, we group all users with 20 or more {feature_title} into the same group.</p>
        """
    })

    fig_dict['activity_counts'] = activity_figs

# Clustering

In [ ]:
# To give extra weight to host admins, add an additional column to the dataframe with the sum of host admin activities and collective admin activities

active_users_df['host_admin_acivity_modifier'] = active_users_df['host_admin_activities'] + active_users_df['collective_admin_activities']

# If user has less than 25 host admin activities, set modifier to 0
active_users_df.loc[active_users_df['host_admin_activities'] < 25, 'host_admin_acivity_modifier'] = 0

# Scale the modifier with Yeo-Johnson Transformation
active_users_df['host_admin_acivity_modifier_yeojohnson'] = pt_std.fit_transform(active_users_df[['host_admin_acivity_modifier']])

# Note that we add host_admin_acivity_modifier_yeojohnson to the list of features to be used for clustering, giving host admins extra weight

clustering_cols = ["expense_activities", "expense_to_own_collective_activities", "host_admin_activities", "collective_admin_activities", "direct_contributions", "collective_contributions", "contributions_to_own_collective", "contributions_via_host", "organization_contributions", "event_orders", "host_admin_acivity_modifier"]
clustering_features = [c + '_yeojohnson' for c in clustering_cols]

# Create a dataframe with the features to be used for clustering
cluster_df = active_users_df[clustering_features].copy()


# Perform k-means clustering on active users

In [ ]:
# Perform k-means clustering on users in active_users_df

# Use the elbow method to determine the optimal number of clusters

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Create a list of the number of clusters to try
k_list = list(range(1, 20))

# Create a list to store the inertia values
inertia_list = []

# For each value of k, perform k-means clustering and calculate the inertia
for k in k_list:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=1000)
    kmeans.fit(cluster_df)
    inertia_list.append(kmeans.inertia_)

# Plot the inertia values for each value of k
plt.plot(k_list, inertia_list, '-o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.xticks(k_list)
plt.show()


In [ ]:
# Choosing 13 clusters because even though the elbow is at 11, trial and error with domain knowledge showed that 13 clusters gives better results
nc = 13

# Perform k-means clustering
kmeans = KMeans(n_clusters=nc, random_state=42, n_init=10, max_iter=1000)
kmeans.fit(cluster_df)

# Add the cluster labels to the dataframe
cluster_df['k_means_cluster'] = kmeans.labels_

# add the cluster labels to the original dataframe
active_users_df['k_means_cluster'] = kmeans.labels_ +1
active_users_df['cluster'] = active_users_df['k_means_cluster']
clusters = active_users_df['cluster']

# Analyze clusters

In [ ]:
# Set temporary cluster names

cluster_names = {}
for c in range(1,clusters.max()+1):
    cluster_names[c] = f'Cluster {c}'
    
active_users_df['cluster_name'] = active_users_df['cluster'].map(cluster_names)

In [ ]:
clustering_cols = ["expense_activities", "expense_to_own_collective_activities", "host_admin_activities", "collective_admin_activities", "direct_contributions", "collective_contributions", "contributions_to_own_collective", "contributions_via_host", "organization_contributions", "event_orders"]

# Display cluster characteristics

# For each cluster in active_users_df, plot all rank histograms in a single image for that cluster
for cluster_num in range(1, clusters.max()+1):
    print(f'Cluster {cluster_num}')

    # print the number of users in the cluster
    print(f'Number of users in cluster: {active_users_df[active_users_df["cluster"] == cluster_num].shape[0]}')
    
    # get the cluster dataframe
    cluster = active_users_df[active_users_df['cluster'] == cluster_num]
    
    # Calculate the number of rows needed based on the length of cols
    num_rows = int(np.ceil(len(cols) / 3))
    
    # Create subplots for this cluster
    fig, axes = plt.subplots(num_rows, 2, figsize=(15, 5*num_rows))  # Adjust the figsize to your needs

    # Flatten axes for easy indexing
    axes = axes.ravel()

    # For each column, plot the histogram
    for i, c in enumerate(clustering_cols):
        # get the non-zero values
        feature = cluster[cluster[c] > 0][c]
        
        # get the labels
        labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10+']
        
        # adjust the bins - note the slight shift for the 10 to ensure it's included in the '10+' bin
        bins = list(range(1, 11)) + [np.inf]
        
        # plot the histogram with a custom histogram calculation to ensure 10+ is handled correctly
        hist_vals, _ = np.histogram(feature, bins=bins)
        
        axes[i].bar(labels, hist_vals, edgecolor='black')
        
        # set the title
        axes[i].set_title(c)
        # set the x-axis label
        axes[i].set_xlabel(f'Number of {c}')
        # set the y-axis label
        axes[i].set_ylabel('Number of Users')

    # Adjust layout and show the combined plots for this cluster
    plt.tight_layout()

    # Display cluster name as title
    plt.suptitle(cluster_names[cluster_num], fontsize=40)
    # padding between the title and the plots
    plt.subplots_adjust(top=0.90)

    plt.show()


# Set cluster names

In [ ]:
# Name clusters based on their characteristics
# This needs to be edited after the clusters have been analyzed

# Create a dictionary to store the cluster names, short and full, and the activity count that is most important for that cluster
cluster_names = {}
cluster_names[1] = ('dir_cont_single', 'One Time Contributors')
cluster_names[2] = ('col_ad_basic', 'Basic Collective Admins')
cluster_names[3] = ('host_ad_basic', 'Basic Host Admins')
cluster_names[4] = ('event_cont', 'Event Orders')
cluster_names[5] = ('col_ad_cont_self', 'Collective Admins Contribute to Own Collectives')
cluster_names[6] = ('org_contr', 'Organization Contributors')
cluster_names[7] = ('exp_sub', 'Expense Submitters')
cluster_names[8] = ('col_ad_cont_via_col', 'Collective Admins with Contributions via Collective')
cluster_names[9] = ('host_ad_high_acti', 'Highly Active Host Admins')
cluster_names[10] = ('col_ad_exp_self', 'Collective Admins Expenses to Own Collectives')
cluster_names[11] = ('host_ad_mod_act', 'Moderately Active Host Admins')
cluster_names[12] = ('dir_contr_repeat', 'Repeated Direct Contributors')
cluster_names[13] = ('host_ad_low_act', 'Low Activity Host Admins')

# Add cluster name shorthand and full name to the dataframe
active_users_df['cluster_name'] = active_users_df['cluster'].map(lambda x: cluster_names[x][0])
active_users_df['cluster_full_name'] = active_users_df['cluster'].map(lambda x: cluster_names[x][1])

In [ ]:
# For each cluster in cluster_name of active_users_df, calculate the mean log norm score of rows in active_users_df for each activity type

activity_features = [f + '_norm_log' for f in clustering_cols]

# Create a dataframe to store the median scores
cluster_mean_scores = pd.DataFrame(columns=activity_features)

# For each cluster, calculate the median score for each activity type
for cluster_num in range(1,clusters.max()+1):
    cluster = active_users_df[active_users_df['cluster'] == cluster_num]
    cluster_mean_scores.loc[cluster_num] = cluster[activity_features].mean()

# Add the cluster names to the dataframe
cluster_mean_scores['cluster_name'] = cluster_mean_scores.index.map(lambda x: cluster_names[x][0])
cluster_mean_scores['cluster_full_name'] = cluster_mean_scores.index.map(lambda x: cluster_names[x][1])

# add count of users in each cluster to the dataframe cluster_mean_scores matching on cluster_name
cluster_mean_scores['count'] = cluster_mean_scores['cluster_name'].map(active_users_df.groupby('cluster_name')['user_id'].count())

# Reorder the columns
cluster_mean_scores = cluster_mean_scores[['cluster_name', 'cluster_full_name' , 'count'] + activity_features]

In [ ]:
order = ['dir_cont_single', 'dir_contr_repeat', 'event_cont', 'org_contr', 'col_ad_basic', 'col_ad_cont_self', 'col_ad_exp_self', 'col_ad_cont_via_col', 'host_ad_basic', 'host_ad_mod_act', 'host_ad_low_act', 'host_ad_high_acti', 'exp_sub']
# sort rows in cluster_mean_scores by cluster_name in same order as order
cluster_mean_scores['cluster_name'] = pd.Categorical(cluster_mean_scores['cluster_name'], order)
cluster_mean_scores = cluster_mean_scores.sort_values('cluster_name')

In [ ]:
import plotly.graph_objects as go
from plotly.express.colors import sample_colorscale

custom_titles = [
    "Expense<br>to other", 
    "Expense<br>to own", 
    "Host admin", 
    "Collective admin", 
    "Direct<br>contributions", 
    "Collective<br>contributions", 
    "Contribute<br>to own", 
    "Contributions<br>via host", 
    "Organization<br>contributions", 
    "Event Orders"
    ]

# for all cluster also create separate plots

user_cluster_figs = []

for idx, (index, row) in enumerate(cluster_mean_scores.iterrows(), start=1):

    fig = go.Figure()

    c = sample_colorscale('turbo', len(cluster_mean_scores)+1)

    trace = go.Scatterpolar(
        r=row[3:].tolist(),
        theta=custom_titles,
        fill='toself',
        name=row['cluster_name'],
        showlegend=False,
        marker = dict(color = c[idx])
    )
    fig.add_trace(trace)

    # Update radial axis to not show tick labels and set the range for each subplot
    fig.update_layout({
        'polar': dict(
            radialaxis=dict(showticklabels=False, range=[0, 0.75])
        )
    })

    # Update layout to adjust the size of the figure based on the number of clusters and reduce margins
    fig.update_layout(
        height=600,
        width=600,
        margin=dict(t=80, b=80, r=100, l=100)
    )

    # Show plot
    fig.show()

    cluster_fig = {
        'fig': fig,
        'title': f'{row["cluster_full_name"]}',
        'description': f'Average activity scores for {row["cluster_full_name"]} cluster. <br> There are <b>{row["count"]} users</b> in this cluster.',
        'info': f""""""
    }

    user_cluster_figs.append(cluster_fig)

fig_dict['user_clusters'] = user_cluster_figs

In [ ]:
# get 10 rows from active_users_df where collective_admin_activities > 500 and less than 1000
active_users_df[(active_users_df['collective_admin_activities'] > 10) & (active_users_df['collective_admin_activities'] < 20)].head(10)

# 

# Save active_users_df to pickle file

In [ ]:
# Save active_users_df to pkl file
active_users_df.to_pickle('active_users_df.pkl')

# save fig_dict to pkl file
with open('fig_dict.pkl', 'wb') as f:
    pickle.dump(fig_dict, f)